
## Analytic Solutions

Underworld provides a set of analytic solutions to Stokes flow problems.

In this example, analytic solution objects are used to configure the analogous numerical system, with the numerical solution then compared to the exact. 

In [1]:
view_soln  = "all"
view_order = 1
view_res   = 32
view_show  = False

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
import os
os.environ["GLUCIFER_USE_XVFB"]="1"

In [3]:
import underworld as uw
import glucifer
from underworld import function as fn
import math
import numpy as np
import collections

In [4]:
# Find all available solutions. 
# Use ordered dict to preserve alphabetical ordering
solns_avail = collections.OrderedDict()
for soln_name in dir(fn.analytic):
    if soln_name[0] == "_": continue  # if private member, ignore
    # get soln class
    soln = getattr(fn.analytic,soln_name)
    # check if actually soln
    if issubclass(soln, fn.analytic._SolBase):
        # construct
        if (view_soln=="all") or view_soln==soln_name[3:]:
            print("Solution added: {}".format(soln_name))
            solns_avail[soln_name] = soln()

Solution added: SolA
Solution added: SolB
Solution added: SolC
Solution added: SolCx
Solution added: SolDA
Solution added: SolDB2d
Solution added: SolDB3d
Solution added: SolH
Solution added: SolKx
Solution added: SolKz
Solution added: SolM
Solution added: SolNL


In [5]:
def get_numerical( soln, res=32, order=1, itol=1.e-6, otol=1.e-6 ):
    '''
    Creates the numerical system corresponding to the provided analytic system.
    
    Parameters
    ----------
    soln : uw.function.analytic._SolBase
        The analytic system
    res : int
        System resolution. Same resolution is used for each axis.
    order : int
        System numerical order. 
    itol : float
        Inner solve tolerance.
    otol : float
        Outer solve tolerance.

        
    '''
    if order == 1:
        els = "Q1/dQ0"
    elif order == 2:
        els = "Q2/dPc1"
    else:
        raise ValueError("Provided system order should be 1 or 2.")

    dim = soln.dim
    mesh  = uw.mesh.FeMesh_Cartesian(elementType=els, elementRes=(res,)*dim,minCoord=(0.,)*dim,maxCoord=(1.,)*dim)
    vel   = uw.mesh.MeshVariable(mesh,dim)
    press = uw.mesh.MeshVariable(mesh.subMesh, 1)
    vel.data[:]   = (0.,)*dim
    press.data[:] = 0.

    bcs = soln.get_bcs(vel)
    visc = soln.fn_viscosity
    if soln.nonlinear==True:
        visc = soln.get_viscosity_nl(vel,press)
    stokes = uw.systems.Stokes(vel, press, fn_viscosity=visc, fn_bodyforce=soln.fn_bodyforce, conditions=[bcs,])
    solver = uw.systems.Solver(stokes)
    if uw.mpi.size==1:
        solver.set_inner_method("lu")
    solver.set_inner_rtol(itol)
    solver.set_outer_rtol(otol)
    
    # if nonlinear, lets first grab a const visc approx soln
    if soln.nonlinear==True:
        stokes.fn_viscosity = 1.
        solver.solve()
        stokes.fn_viscosity = visc

    return mesh, vel, press, solver

In [6]:
for view_soln, soln in solns_avail.items():
    print(view_soln)

    mesh, vel, press, solver = get_numerical( soln, res=view_res, order=view_order  )

    if soln.nonlinear:
        solver.solve(nonLinearIterate=True)
    else:
        solver.solve()
        
    viz_args = {"border":3,
                "figsize":(500,500),
                "margin":0.05
               }
    prop_args = {"resolution":view_res*5,
                 "colourBar" :False}
    if soln.dim == 3:
        prop_args["crossSection"]='z=0.5'

# figure for viscosity
    fig_viscosity = glucifer.Figure(**viz_args)
    fig_viscosity.append(glucifer.objects.Surface(mesh,soln.fn_viscosity,
                                                  colours="lightgrey black", onMesh=False, **prop_args ))
    if view_show:
        print("Viscosity:")
        fig_viscosity.show()
    fig_viscosity.save(view_soln+"_viscosity")

# figure for bodyforce
    bf_mag = fn.math.sqrt(fn.math.dot(soln.fn_bodyforce,soln.fn_bodyforce))
    fig_bodyforce = glucifer.Figure(**viz_args)
    fig_bodyforce.append(glucifer.objects.Surface(mesh, bf_mag,
                                                  colours="white black",     onMesh=False, **prop_args))
    if view_show:
        print("Body Force:")
        fig_bodyforce.show()
    fig_bodyforce.save(view_soln+"_bodyforce")

# figure for velocity
    vel_mag = fn.math.sqrt(fn.math.dot(vel,vel))
    fig_vel = glucifer.Figure(**viz_args)
    fig_vel.append(glucifer.objects.Surface(mesh,vel_mag,
                                                    colours="white black",   onMesh=True, **prop_args))
    if view_show: 
        print("Velocity:")
        fig_vel.show()
    fig_vel.save(view_soln+"_velocity")
    
# figure for velocity error
    vel_mag_err = fn.math.sqrt(fn.math.dot(soln.fn_velocity-vel,soln.fn_velocity-vel))
    fig_vel_error = glucifer.Figure(**viz_args)
    fig_vel_error.append(glucifer.objects.Surface(mesh,vel_mag_err,
                                                    colours="white black",   onMesh=True, **prop_args))
    if view_show:
        print("Velocity Error:")
        fig_vel_error.show()
    fig_vel_error.save(view_soln+"_velocity_error")

# figure for pressure
    fig_press = glucifer.Figure(**viz_args)
    fig_press.append(glucifer.objects.Surface(mesh,press,
                                                  colours="white black",     onMesh=False, **prop_args ))
    if view_show:
        print("Pressure:")
        fig_press.show()
    fig_press.save(view_soln+"_pressure")

# figure for pressure error
    # project to get average error across element
    proj_press_error = mesh.subMesh.add_variable(1)
    press_err = uw.function.math.abs(press - soln.fn_pressure)
    projector = uw.utils.MeshVariable_Projection(proj_press_error,press_err)
    projector.solve()
    fig_press_error = glucifer.Figure(**viz_args)
    fig_press_error.append(glucifer.objects.Surface(mesh,proj_press_error,
                                                    colours="white black",   onMesh=False, **prop_args ))
    if view_show:
        print("Pressure Error:")
        fig_press_error.show()
    fig_press_error.save(view_soln+"_pressure_error")

SolA
SolB
SolC
SolCx
SolDA
SolDB2d


/c/Users/jmansour/uw2_jmansour/underworld/systems/_bsscr.py:477: UserWarning: A PETSc error has been encountered during the solve. Solution fields are possibly erroneous. 

This error is probably due to an incorrectly constructed linear system. Please check that your boundary conditions are consistent and sufficient and that your viscosity is positive everywhere. If you are deforming the mesh, ensure that it has not become tangled. 

The resultant KSPConvergedReasons are (f_hat, outer, backsolve) (0,-3,0).


  warnings.warn(estring)


SolDB3d
SolH
SolKx
SolKz
SolM
SolNL


In [6]:
latex = True
if uw.mpi.rank==0 and latex:
    rows = []
    header = []
    im_options="height=1.4cm,keepaspectratio"
    for solname,solval  in solns_avail.items():
        row = []
        header.append("Model")
        row.append(solname[3:])              
        header.append("$\\eta$")
        row.append("\includegraphics[{0}]{{images/{1}}}".format(im_options,solname+"_viscosity.png"))
        header.append("$\\left| \mathbf{F} \\right|$")
        row.append("\includegraphics[{0}]{{images/{1}}}".format(im_options,solname+"_bodyforce.png"))
        header.append("$\\left| \mathbf{u}^h \\right|$")
        row.append("\includegraphics[{0}]{{images/{1}}}".format(im_options,solname+"_velocity.png"))
        header.append("$\\left| \mathbf{u}-\mathbf{u}^h \\right|$")
        row.append("\includegraphics[{0}]{{images/{1}}}".format(im_options,solname+"_velocity_error.png"))
        header.append("$p^h$")
        row.append("\includegraphics[{0}]{{images/{1}}}".format(im_options,solname+"_pressure.png"))
        header.append("$\\left| p-p^h \\right|$")
        row.append("\includegraphics[{0}]{{images/{1}}}".format(im_options,solname+"_pressure_error.png"))

        rows.append(row)

    import tabulate
    tabulate.LATEX_ESCAPE_RULES={}
    tabstring = tabulate.tabulate(rows, tablefmt='latex_raw',floatfmt=".2f",headers=header)
    tabstring = tabstring.replace("tabular", "tabu")
    tabstring = tabstring.replace("{lllllll}"," to \\textwidth {X[0.5,l,m]X[1,c,m]X[1,c,m]X[1,c,m]X[1,c,m]X[1,c,m]X[1,c,m]}")
    print(tabstring)

\begin{tabu} to \textwidth {X[0.5,l,m]X[1,c,m]X[1,c,m]X[1,c,m]X[1,c,m]X[1,c,m]X[1,c,m]}
\hline
 Model   & $\eta$                                                                       & $\left| \mathbf{F} \right|$                                                  & $\left| \mathbf{u}^h \right|$                                               & $\left| \mathbf{u}-\mathbf{u}^h \right|$                                          & $p^h$                                                                       & $\left| p-p^h \right|$                                                            \\
\hline
 A       & \includegraphics[height=1.4cm,keepaspectratio]{images/SolA_viscosity.png}    & \includegraphics[height=1.4cm,keepaspectratio]{images/SolA_bodyforce.png}    & \includegraphics[height=1.4cm,keepaspectratio]{images/SolA_velocity.png}    & \includegraphics[height=1.4cm,keepaspectratio]{images/SolA_velocity_error.png}    & \includegraphics[height=1.4cm,keepaspectratio]{images/SolA_pressure.png} 